# ROOT C++ Kernel Example

In the <a href="ROOT_Example.ipynb">first example</a> there was some basic Python/PyROOT commands, with `%%cpp` magics to allow us to move back and forth between Python and C/C++-style code.

Recent versions of ROOT also ship a C++ kernel, which allows the entire environment to be based in C++. This is like opening ROOT from the command line and doing everything in ACLiC. So, if you're Python-averse or just more comfortable/knowledgable in C++, this is a good way to go

## Some quick philisophical points and pointers

You may find other uses/reasons for using a notebook, but one of the things I like is that I can execute code blocks, and then re-execute them as I like/more easily.

There's a reason that a lot of notebooks like this work well with Python. Because python is not strongly-typed and more flexible, it's much easier to execute code like this:
```
input_parameter = 50.0
output = RunCoolFunction(input_parameter)
print output
```
over and over and over again, re-executing the same cell, just changing the value of the input parameter.

If we do something similar in C++, it might look like this:
```
float input_parameter = 50.0;
int output = RunCoolFunction(input_parameter);
std::cout << output << std::endl;
```

That will work the first time you execute the cell. But if you re-execute it, you will be encountered with something like:
```
input_line_34:2:8: error: redefinition of 'input_parameter'
 float input_parameter = 50.0;
       ^
input_line_29:2:8: note: previous definition is here
 float input_parameter = 50.0;
```

You have two options:
1. Everytime you want to re-execute, you have to use the "Restart & Run All" option in the "Kernel" menu above.
2. You need to separate cells where you declare things (variables, functions, etc.) from cells you may want to execute many times.

Option 1 is certainly valid, doable, and sometimes the best way to do things anyway to ensure you aren't going to have problems. But, in my opinion, it's worth thinking through things a little to orient your coding style so Option2 is at least an option to you. Let's see that in action here.

In [1]:
//Function to generate n random numbers and add them together
//Input: TRandom3 generaotor, and the number of numbers to add together
//Output: double of the addition of all the numbers.

double AddRandomNumbers(TRandom3 &rng, int const n)
{
    double sum = 0.0;
    for(int i=0; i<n; ++i)
        sum += rng.Rndm();
    return sum;
}

In [2]:
//create random number generator
TRandom3 rng(0);

In [3]:
//Test the function. Execute this cell as often as you want!
AddRandomNumbers(rng,1000)

(double) 507.69172


In [4]:
//Setup a stopwatch to measure the execution time.
TStopwatch timer;

In [5]:
//Measure the real time and CPU time for the function execution.
//Again, we can rerun this cell as often as we want, changing the input parameters.
timer.Start();
AddRandomNumbers(rng,1e8);
timer.Stop();
std::cout << "RealTime: " << timer.RealTime()*1000. << " ms"
          << " CpuTime: " << timer.CpuTime()*1000. << " ms." << std::endl;
std::cout << "Difference  = " << (timer.RealTime()-timer.CpuTime())*1000. << " ms." << std::endl;

RealTime: 683.322 ms CpuTime: 680 ms.
Difference  = 3.32195 ms.


In [6]:
//Let's make an ntuple to store measured time values
TNtuple *nt = new TNtuple("nt","MyTimeNtuple","n_rndm:time");

In [7]:
//Let's setup a for loop to fill it.
nt->Reset();
for(int i_n=0; i_n<10; ++i_n)
{
    int n_times = std::pow(10,i_n);
    timer.Start();
    AddRandomNumbers(rng,n_times);
    timer.Stop();
    nt->Fill(n_times,timer.RealTime());
    std::cout << "\t" << n_times << " :\t\t" << timer.RealTime()*1000. << " ms." << std::endl;
}

	1 :		0.00286102 ms.
	10 :		0.000953674 ms.
	100 :		0.000953674 ms.
	1000 :		0.00715256 ms.
	10000 :		0.0751019 ms.
	100000 :		0.683069 ms.
	1000000 :		6.8388 ms.
	10000000 :		68.3889 ms.
	100000000 :		682.878 ms.
	1000000000 :		6826.96 ms.


In [8]:
//make the canvas just once
TCanvas c("c");

In [9]:
%jsroot on
c.cd();
nt->SetMarkerStyle(20);
nt->Draw("time:n_rndm>>h1");
c.SetLogx(true);
c.SetLogy(true);
c.Draw();

OK, that was fun! But let's do one more thing. We can ask root to explicitly compile the function and run that. It should be faster? Let's try it.

To tell ROOT to get ACLiC to really compile the function, we use the `%%cpp -a` magic. Note also that we need to include the header for TRandom3, otherwise we'll get compilation errors.

In [10]:
%%cpp -a
//Function to generate n random numbers and add them together
//Input: TRandom3 generaotor, and the number of numbers to add together
//Output: double of the addition of all the numbers.

#include "TRandom3.h"

double AddRandomNumbers2(TRandom3 &rng, int const n)
{
    double sum = 0.0;
    for(int i=0; i<n; ++i)
        sum += rng.Rndm();
    return sum;
}

Info in <TUnixSystem::ACLiC>: creating shared library /uboone/app/users/wketchum/notebooks/Sample_Notebooks/a5cb9257_C.so


In [11]:
//Let's make a different ntuple to store new measured time values
TNtuple *nt2 = new TNtuple("nt2","MyTimeNtuple","n_rndm:time");

In [12]:
//Let's setup a for loop to fill it.
nt2->Reset();
for(int i_n=0; i_n<10; ++i_n)
{
    int n_times = std::pow(10,i_n);
    timer.Start();
    AddRandomNumbers2(rng,n_times);
    timer.Stop();
    nt2->Fill(n_times,timer.RealTime());
    std::cout << "\t" << n_times << " :\t\t" << timer.RealTime()*1000. << " ms." << std::endl;
}

	1 :		0.120163 ms.
	10 :		0.00214577 ms.
	100 :		0.000953674 ms.
	1000 :		0.00691414 ms.
	10000 :		0.056982 ms.
	100000 :		0.593901 ms.
	1000000 :		5.69606 ms.
	10000000 :		57.0438 ms.
	100000000 :		573.081 ms.
	1000000000 :		5693.79 ms.


In [13]:
%jsroot on
c.cd();
nt->Draw("time:n_rndm");
nt2->SetMarkerStyle(20);
nt2->SetMarkerColor(kRed);
nt2->Draw("time:n_rndm","1","same");
c.SetLogx(true);
c.SetLogy(true);
c.Draw();